Note: Oct 2 Adding support for dates, the rest of the ipyn still need to be edited to allow for this. 


In [1]:
from  bs4 import BeautifulSoup
import requests
import pandas as pd
import re


In [2]:
att_iphones_url = "https://swappa.com/mobile/att/iphones"

models = ['apple-iphone-x-a1865','apple-iphone-se', 'apple-iphone-8-plus-a1864', 'apple-iphone-8-a1863',
          'apple-iphone-7-plus-a1661','apple-iphone-7-a1660','apple-iphone-6s-plus','apple-iphone-6s']

carriers = ['att']

In [3]:
def pull_sold_data(link):
    url = 'https://swappa.com' + link
    page = requests.get(url).text
    soup = BeautifulSoup(page, features="html5lib")
    recently_sold = soup.findAll(class_='table table-bordered table-condensed table-centered')
    date = re.sub('\s+', '', recently_sold[0].findAll('td')[6].text)
    return(date)

In [4]:
def sale_details(url):
    
    page = requests.get(url).text
    soup = BeautifulSoup(page, features="html5lib")
    recently_sold = soup.findAll(class_='listing_row listing_ listing_')
    
    phone_data = []
    for i in range(len(recently_sold)-3):
        condition = recently_sold[i].find(class_="condition_label").text
        color = recently_sold[i].find(class_="color_label").text
        storage = recently_sold[i].find(class_="storage_label").text
        sold_price = recently_sold[i].find(class_="price").text
        
        link = recently_sold[i].find('a')['href']
        date = pull_sold_data(link)
    
        phone_data.append([condition,color,int(storage.strip('GB')),int(sold_price.strip('$')),date])
        
    return(phone_data)

In [5]:
print(sale_details("https://swappa.com/buy/apple-iphone-x-a1865-att"))

[['Mint', 'Silver', 256, 775, 'Oct09,2018-10:15PM'], ['Good', 'Gray', 64, 660, 'Oct05,2018-07:10PM']]


In [6]:
col_names = ['Phone Model', 'Carrier', 'Condition', 'Color', 'Size', 'Recently Sold Price','Date']
iphone_df = pd.DataFrame(columns=col_names)
print(iphone_df)

Empty DataFrame
Columns: [Phone Model, Carrier, Condition, Color, Size, Recently Sold Price, Date]
Index: []


In [7]:
for model in models:
    for carrier in carriers:
        url = "https://swappa.com/buy/"+ model + '-' + carrier
        data = sale_details(url)
            
        for sold in data:
            iphone_df.loc[len(iphone_df)] = [model,carrier,sold[0],sold[1],sold[2],sold[3],sold[4]]


In [8]:
print(iphone_df.head(10))
iphone_df.shape

                 Phone Model Carrier Condition      Color Size  \
0       apple-iphone-x-a1865     att      Mint     Silver  256   
1       apple-iphone-x-a1865     att      Good       Gray   64   
2            apple-iphone-se     att      Good       Grey   32   
3            apple-iphone-se     att      Good       Grey  128   
4  apple-iphone-8-plus-a1864     att      Good       Gray   64   
5  apple-iphone-8-plus-a1864     att      Mint       Gray  256   
6       apple-iphone-8-a1863     att      Mint     Silver  256   
7       apple-iphone-8-a1863     att      Mint        Red   64   
8  apple-iphone-7-plus-a1661     att      Fair  Rose Gold  128   
9  apple-iphone-7-plus-a1661     att      Good      Black   32   

  Recently Sold Price                Date  
0                 775  Oct09,2018-10:15PM  
1                 660  Oct05,2018-07:10PM  
2                 139  Oct10,2018-12:27PM  
3                 175  Oct10,2018-08:23AM  
4                 549  Oct09,2018-10:42AM  
5        

(16, 7)

In [9]:
file_name = "iphone_df_Oct_10_examples.csv"
iphone_df.to_csv(file_name, encoding='utf-8', index=False)